In [ ]:
import pandas as pd
import regex # needs to be installed with pip
import re
import string
from collections import Counter
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Ensemble Learning/cyberbullying_tweets.csv')
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [ ]:
# introduce temporary column to seperate words
df["list_of_words"] = df['tweet_text'].apply(lambda x: x.split()) 
df.head()

,tweet_text,cyberbullying_type,list_of_words
0,"In other words #katandandre, your food was cra...",not_cyberbullying,"[In, other, words, #katandandre,, your, food, ..."
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,"[Why, is, #aussietv, so, white?, #MKR, #theblo..."
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,"[@XochitlSuckkks, a, classy, whore?, Or, more,..."
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,"[@Jason_Gio, meh., :P, thanks, for, the, heads..."
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,"[@RudhoeEnglish, This, is, an, ISIS, account, ..."


In [ ]:
# count length of tweet
df["characters per tweet"] = df['tweet_text'].apply(lambda x: len(x))
df["words per tweet"] = df['tweet_text'].apply(lambda x: len(x.split()))
df.head()

,tweet_text,cyberbullying_type,list_of_words,characters per tweet,words per tweet
0,"In other words #katandandre, your food was cra...",not_cyberbullying,"[In, other, words, #katandandre,, your, food, ...",61,9
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,"[Why, is, #aussietv, so, white?, #MKR, #theblo...",115,14
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,"[@XochitlSuckkks, a, classy, whore?, Or, more,...",60,9
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,"[@Jason_Gio, meh., :P, thanks, for, the, heads...",103,18
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,"[@RudhoeEnglish, This, is, an, ISIS, account, ...",103,18


In [ ]:
# identify word shape: 
# all upper
df["nb_upper"] = df["list_of_words"].apply(lambda x: sum(1 for c in x if c.isupper()))
# all lower
df["nb_lower"] = df["list_of_words"].apply(lambda x: sum(1 for c in x if c.islower()))

# capitalized  :first char upper, others lower 
count_capitalized  = lambda x: sum([(a[0].isupper() and a[1:].islower()) for a in x if len(a)>1])
df["nb_capitalized"] = df["list_of_words"].apply(count_capitalized)


# camelcase https://www.dictionary.com/browse/camelcase
# simplification = word element beginning with an uppercase letter that is not in nb_upper and not in
# nb_capitalized 
count_camel_case = lambda x: sum([(a[0].isupper() and
                                       (not a[1:].islower() and not a[1:].isupper())) for a in x if len(a)>1])
df["mixed_upper_lower_not_capitalized"] = df["list_of_words"].apply(count_camel_case)

# count how many words have the length x+1 for x in range(20)
for i in range(20):
    count_len = lambda x: sum([len(a)==i+1 for a in x])
    df[f"nb_len_{i+1}"] = df["list_of_words"].apply(count_len)
    
# remove the help column 
df.drop("list_of_words", axis=1,inplace=True)


In [ ]:
# df_all_orgas = df_all_orgas.reset_index(drop=True)

In [ ]:
df.head() 

,tweet_text,cyberbullying_type,characters per tweet,words per tweet,nb_upper,nb_lower,nb_capitalized,mixed_upper_lower_not_capitalized,nb_len_1,nb_len_2,nb_len_3,nb_len_4,nb_len_5,nb_len_6,nb_len_7,nb_len_8,nb_len_9,nb_len_10,nb_len_11,nb_len_12,nb_len_13,nb_len_14,nb_len_15,nb_len_16,nb_len_17,nb_len_18,nb_len_19,nb_len_20
0,"In other words #katandandre, your food was cra...",not_cyberbullying,61,9,0,8,1,0,0,1,1,3,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,115,14,1,9,1,0,0,2,1,2,0,2,0,1,3,0,1,0,0,1,1,0,0,0,0,0
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,60,9,0,7,1,0,1,1,1,1,0,3,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,103,18,1,16,0,0,0,2,6,2,3,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,103,18,1,12,4,0,1,6,1,3,1,1,2,1,0,1,0,0,0,1,0,0,0,0,0,0


In [ ]:
df.columns

Index(['tweet_text', 'cyberbullying_type', 'characters per tweet',
       'words per tweet', 'nb_upper', 'nb_lower', 'nb_capitalized',
       'mixed_upper_lower_not_capitalized', 'nb_len_1', 'nb_len_2', 'nb_len_3',
       'nb_len_4', 'nb_len_5', 'nb_len_6', 'nb_len_7', 'nb_len_8', 'nb_len_9',
       'nb_len_10', 'nb_len_11', 'nb_len_12', 'nb_len_13', 'nb_len_14',
       'nb_len_15', 'nb_len_16', 'nb_len_17', 'nb_len_18', 'nb_len_19',
       'nb_len_20'],
      dtype='object')

In [ ]:
df_all_orgas['text'].str.lower()

1340367556947943425    a $20,000 fee to change a wedding date? new bi...
1340329823282016256    u.s. intelligence agencies are increasingly fo...
1340307168923414531    rebuilding trust in the justice department sta...
1340141062724194306    biden says lindsey graham is a 'personal disap...
1340065561523458048    75 people and counting test positive for covid...
                                             ...                        
1316355491946213376    your dog has a mid-life crisis too! pet pooche...
1316314715434430464    thai pro-democracy protesters fight with royal...
1316309906010583045    fisherman is filmed holding dolphin calf under...
1316291670217052166    thai pro-democracy protesters fight with royal...
1316036386554884096    12,000lb raf 'tallboy' bomb explodes while bei...
Name: text, Length: 20796, dtype: object

In [ ]:
letters = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z",
           str(0),"1","2","3","4","5","6","7","8","9","?","!","@","$","&","#","%","(",")","-","+","=","/"]

In [ ]:
lst = []
for index, row in df.iterrows(): 
    chars_in_string = Counter(row["tweet_text"])
    res = []
    for letter in letters:
        if letter in chars_in_string:
            res.append(chars_in_string[letter])
        else: 
            res.append(0) 
    lst.append(res)

df_temp = pd.DataFrame(lst, columns=letters, index=df.index)

In [ ]:
df_temp.head()

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,0,1,2,3,4,5,6,7,8,9,?,!,@,$,&,#,%,(,),-,+,=,/
0,5,0,2,4,2,1,0,1,3,0,2,1,1,3,6,1,0,6,3,2,2,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0
1,3,3,2,4,10,0,1,4,7,0,1,3,1,4,6,0,0,4,8,7,4,1,1,0,3,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,10,0,0,0,0,0,0,0
2,3,0,5,1,6,0,0,2,1,0,4,3,1,0,3,1,0,4,4,2,2,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0
3,6,2,2,4,8,1,1,5,2,0,1,0,1,8,10,1,0,5,3,10,4,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,6,1,4,3,6,0,2,4,9,0,1,5,1,6,4,1,0,2,7,5,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = df.join(df_temp)

In [ ]:
df.head()

,tweet_text,cyberbullying_type,characters per tweet,words per tweet,nb_upper,nb_lower,nb_capitalized,mixed_upper_lower_not_capitalized,nb_len_1,nb_len_2,nb_len_3,nb_len_4,nb_len_5,nb_len_6,nb_len_7,nb_len_8,nb_len_9,nb_len_10,nb_len_11,nb_len_12,nb_len_13,nb_len_14,nb_len_15,nb_len_16,nb_len_17,nb_len_18,nb_len_19,nb_len_20,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,0,1,2,3,4,5,6,7,8,9,?,!,@,$,&,#,%,(,),-,+,=,/
0,"In other words #katandandre, your food was cra...",not_cyberbullying,61,9,0,8,1,0,0,1,1,3,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,5,0,2,4,2,1,0,1,3,0,2,1,1,3,6,1,0,6,3,2,2,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,115,14,1,9,1,0,0,2,1,2,0,2,0,1,3,0,1,0,0,1,1,0,0,0,0,0,3,3,2,4,10,0,1,4,7,0,1,3,1,4,6,0,0,4,8,7,4,1,1,0,3,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,10,0,0,0,0,0,0,0
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,60,9,0,7,1,0,1,1,1,1,0,3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,3,0,5,1,6,0,0,2,1,0,4,3,1,0,3,1,0,4,4,2,2,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,103,18,1,16,0,0,0,2,6,2,3,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,6,2,2,4,8,1,1,5,2,0,1,0,1,8,10,1,0,5,3,10,4,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,103,18,1,12,4,0,1,6,1,3,1,1,2,1,0,1,0,0,0,1,0,0,0,0,0,0,6,1,4,3,6,0,2,4,9,0,1,5,1,6,4,1,0,2,7,5,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
lst = []

for index, row in df.iterrows(): 
    non_ASCII = 0
    for c in row["tweet_text"]:
        if 0 <= ord(c) <= 127:
            pass
        else:
            non_ASCII += 1
            # this is a non-ascii character.
     
    lst.append(non_ASCII)

df_temp = pd.DataFrame({'non_ASCII':lst},index=df.index)

In [ ]:
df_temp.head()

,non_ASCII
0,0
1,0
2,0
3,0
4,0


In [ ]:
df = df.join(df_temp)

In [ ]:
df.head(20)

,tweet_text,cyberbullying_type,characters per tweet,words per tweet,nb_upper,nb_lower,nb_capitalized,mixed_upper_lower_not_capitalized,nb_len_1,nb_len_2,nb_len_3,nb_len_4,nb_len_5,nb_len_6,nb_len_7,nb_len_8,nb_len_9,nb_len_10,nb_len_11,nb_len_12,nb_len_13,nb_len_14,nb_len_15,nb_len_16,nb_len_17,nb_len_18,nb_len_19,nb_len_20,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,0,1,2,3,4,5,6,7,8,9,?,!,@,$,&,#,%,(,),-,+,=,/,non_ASCII
0,"In other words #katandandre, your food was cra...",not_cyberbullying,61,9,0,8,1,0,0,1,1,3,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,5,0,2,4,2,1,0,1,3,0,2,1,1,3,6,1,0,6,3,2,2,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,115,14,1,9,1,0,0,2,1,2,0,2,0,1,3,0,1,0,0,1,1,0,0,0,0,0,3,3,2,4,10,0,1,4,7,0,1,3,1,4,6,0,0,4,8,7,4,1,1,0,3,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,10,0,0,0,0,0,0,0,0
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,60,9,0,7,1,0,1,1,1,1,0,3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,3,0,5,1,6,0,0,2,1,0,4,3,1,0,3,1,0,4,4,2,2,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,103,18,1,16,0,0,0,2,6,2,3,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,6,2,2,4,8,1,1,5,2,0,1,0,1,8,10,1,0,5,3,10,4,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,103,18,1,12,4,0,1,6,1,3,1,1,2,1,0,1,0,0,0,1,0,0,0,0,0,0,6,1,4,3,6,0,2,4,9,0,1,5,1,6,4,1,0,2,7,5,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,"@Raja5aab @Quickieleaks Yes, the test of god i...",not_cyberbullying,131,23,0,20,1,0,0,7,4,5,1,1,0,0,2,1,1,0,1,0,0,0,0,0,0,0,9,2,2,6,14,3,3,3,8,1,2,3,0,3,10,1,0,8,7,9,1,2,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0
6,Itu sekolah ya bukan tempat bully! Ga jauh kay...,not_cyberbullying,54,10,0,8,2,0,0,2,1,2,1,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,10,2,0,0,3,0,0,2,0,1,4,3,1,2,1,1,0,1,1,3,4,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,Karma. I hope it bites Kat on the butt. She is...,not_cyberbullying,63,14,1,10,3,0,1,3,3,3,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,4,0,0,3,3,1,1,0,2,2,2,1,0,2,4,8,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8,@stockputout everything but mostly my priest,not_cyberbullying,44,6,0,6,0,0,0,1,1,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,3,0,1,1,2,0,1,1,2,1,3,2,0,2,3,7,3,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9,Rebecca Black Drops Out of School Due to Bully...,not_cyberbullying,50,9,0,2,7,0,0,2,2,0,2,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2,1,4,0,3,1,1,1,1,0,1,4,0,1,5,1,0,1,1,2,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.to_csv('/content/drive/MyDrive/Ensemble Learning/style_data.csv')

In [ ]:
df_all_orgas["0"].head()

1340367556947943425    5
1340329823282016256    2
1340307168923414531    1
1340141062724194306    0
1340065561523458048    0
Name: 0, dtype: int64